In [1]:
import pandas
from pandas import *
import numpy
from datetime import datetime

# Get the training data
df_train = pandas.read_csv("data/train.csv.gz")

# Only small sample for now
#
df_train = df_train.sample(100000)
# Reading test data set
df_test = pandas.read_csv("data/test.csv.gz")

if False:
    df_train = pandas.read_csv("../input/train.csv")
    df_test = pandas.read_csv("../input/test.csv")



print("Starting with {0}".format(df_train.count()))
display(df_train.sample(10))
df_train.describe()

Starting with date     100000
store    100000
item     100000
sales    100000
dtype: int64


,date,store,item,sales
889565,2013-10-31,8,49,37
596460,2016-03-30,7,33,43
319016,2016-07-16,5,18,106
256625,2015-09-13,1,15,83
742103,2015-01-18,7,41,17
55815,2015-11-02,1,4,16
402676,2015-08-15,1,23,31
729058,2014-04-30,10,40,27
265214,2014-03-21,6,15,47
296547,2015-01-06,3,17,22


,store,item,sales
count,100000.000000,100000.000000,100000.000000
mean,5.502020,25.568740,52.307270
std,2.870484,14.404239,28.922297
min,1.000000,1.000000,2.000000
25%,3.000000,13.000000,30.000000
50%,5.000000,26.000000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,209.000000


In [2]:
# Adding various information to the dataframe for later 
# DayOfWeek
# DayOfMonth
# Month
# Year
# CalendarWeek


# Add all variables that we would like to this model

# Extracting some variable from date
def date_info(date):
    datetime_object = datetime.strptime(date, "%Y-%m-%d")
    
    week_day = datetime_object.weekday()
    cw = datetime_object.isocalendar()[1]
    
    day = datetime_object.day
    month = datetime_object.month
    year = datetime_object.year
    
    days = (datetime_object - datetime(2010,1,1)).days
    
    return (week_day, cw, day, days, month, year)


def extrapolate(df):
    df['week_day'], df['cw'], df['day'], df['days'], df['month'], df['year'] = zip(*df["date"].map(date_info))

    # Set type of store and item to category
    df['item'] = df['item'].astype('category')
    df['store'] = df['store'].astype('category')
    
    
    # Convert date as proper time as well
    df['time'] = df['date'].astype('datetime64[ns]')
    
    # Month should be a category as well
    df['week_day'] = df['week_day'].astype('category')
    df['month'] = df['month'].astype('category')

    return df

# Add those information to df_train and df_test
df_train = extrapolate(df_train)
df_test = extrapolate(df_test)

# Understand how scale change depending on dow
df_dow = df_train
display(df_train.sample(10))


,date,store,item,sales,week_day,cw,day,days,month,year,time
469030,2017-04-24,7,26,32,0,17,24,2670,4,2017,2017-04-24
102412,2013-06-06,7,6,43,3,23,6,1252,6,2013,2013-06-06
495972,2016-02-01,2,28,75,0,5,1,2222,2,2016,2016-02-01
815942,2017-03-27,7,45,44,0,13,27,2642,3,2017,2017-03-27
795568,2016-06-12,6,44,23,6,23,12,2354,6,2016,2016-06-12
162437,2017-10-16,9,9,36,0,42,16,2845,10,2017,2017-10-16
475408,2014-10-11,1,27,14,5,41,11,1744,10,2014,2014-10-11
711367,2015-11-20,10,39,55,4,47,20,2149,11,2015,2015-11-20
481429,2016-04-06,4,27,16,2,14,6,2287,4,2016,2016-04-06
199629,2014-08-19,10,11,68,1,34,19,1691,8,2014,2014-08-19


In [3]:
# Generate our training/validation datasets
from sklearn import model_selection

result_cols = ['sales']
input_cols = ['store', 'item', 'week_day', 'cw', 'day', 'days', 'month', 'year']

def get_values(df, cols=[]):
    final_df = df
    
    print(final_df.columns.values)
    # Remove all columns that are not inside the list
    for column in final_df.columns.values:
        if column not in cols:
            final_df = final_df.drop(columns=[column])
    return final_df.values


X = get_values(df_train, input_cols)
Y = get_values(df_train, result_cols).ravel()
X_test = get_values(df_test, input_cols)

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Create validation + training set

['date' 'store' 'item' 'sales' 'week_day' 'cw' 'day' 'days' 'month' 'year'
 'time']
['date' 'store' 'item' 'sales' 'week_day' 'cw' 'day' 'days' 'month' 'year'
 'time']
['id' 'date' 'store' 'item' 'week_day' 'cw' 'day' 'days' 'month' 'year'
 'time']


In [4]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor



# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

# Spot Check Algorithms
models = []
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('TREEREG', DecisionTreeRegressor()))

#models.append(('NB', GaussianNB()))
#models.append(('NEURAL', MLPClassifier()))
#models.append(('LR', LogisticRegression()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []

import sklearn
print(sklearn.__version__)


# Simple function to represent the result
def mean_absolute_percentage_error(Y_validation, Y_validation_predict):
    result = 0
    for i in range(0, len(Y_validation)):
        result += (abs(Y_validation[i] - Y_validation_predict[i]))/(abs(Y_validation[i])+abs(Y_validation_predict[i]))
    return result / len(Y_validation) * 200
print("Starting all models")
#Y_train=Y_train.astype('int')

max_score = 10000
for name, model in models:
    print("Executing for model {0}".format(name))
    #kfold = model_selection.KFold(n_splits=10, random_state=seed)
    #cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    #results.append(cv_results)
    #names.append(name)
    #msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    #print(msg)
    
    # Writing ht
    
    # Training the model
    model.fit(X_train, Y_train)
    
    Y_validation_predict = model.predict(X_validation)
    score = mean_absolute_percentage_error(Y_validation, Y_validation_predict)
    print("Model {0} got score of {1}".format(name, score))
    
    Y_test = model.predict(X_test)
    print("Result should be {0}".format(Y_test))

    # Create new result dataframe:
    result_df = df_test['id']
    df_test['sales'] = Y_test
    df_test.head(5)
    df_result = df_test[['id', 'sales']]

    # Write the dataframe as output
    df_result.head(5)

    df_result.to_csv("output_{0}.csv".format(name), index=False)
    if max_score > score:
        max_score = score
        print("Choosing {0} as best result".format(name))
        df_result.to_csv("output_best.csv".format(name), index=False)
print("Final result should be around {0}".format(max_score))


0.19.2
Starting all models
Executing for model CART
Model CART got score of 27.308117278357358
Result should be [18 18 18 ..., 82 82 82]
Choosing CART as best result
Executing for model TREEREG
Model TREEREG got score of 24.137605823241316
Result should be [ 20.50310559  20.50310559  20.50310559 ...,  74.68493151  74.68493151
  74.68493151]
Choosing TREEREG as best result
Final result should be around 24.137605823241316
